In [1]:
import torch
import torch.nn as nn
import torch.utils as utils
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms

In [2]:
epoch = 1
batch_size = 16
learning_rate = 1e-3

In [3]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

Processing...
Done!


In [ ]:
epoch = 1
batch_size =16
learning_rate = 0.001

# Download Data

mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test  = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

# Check the datasets downloaded

print(mnist_train.__len__())
print(mnist_test.__len__())
img1,label1 = mnist_train.__getitem__(0)
img2,label2 = mnist_test.__getitem__(0)

print(img1.size(), label1)
print(img2.size(), label2)

# Set Data Loader(input pipeline)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test,batch_size=batch_size,shuffle=True)

In [33]:
class CNN(nn.Module):

    def __init__(self):

        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1,16,5),   # batch x 16 x 24 x 24
                        nn.ReLU(),
                        nn.BatchNorm2d(16),
                        nn.Conv2d(16,32,5),  # batch x 32 x 20 x 20
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.MaxPool2d(2,2)   # batch x 32 x 10 x 10
        )
        self.layer2 = nn.Sequential(
                        nn.Conv2d(32,64,5),  # batch x 64 x 6 x 6
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64,128,5),  # batch x 128 x 2 x 2
                        nn.ReLU()
        )
        self.fc = nn.Linear(2*2*128,10)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(batch_size, -1)
        out = self.fc(out)
        return out
        
cnn = CNN()

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

In [34]:
for i in range(epoch):
    for j,[image,label] in enumerate(train_loader):
        image = Variable(image).cuda()
        label = Variable(label).cuda()
        cnn.cuda()
        
        optimizer.zero_grad()
        result = cnn.forward(image)
        loss = loss_func(result,label)
        loss.backward()
        optimizer.step()
        
        if j % 100 == 0:
            print(loss)

# Test with test data
# In order test, we need to change model mode to .eval()
# and get the highest score label for accuracy
# Change model to 'eval' mode (BN uses moving mean/var)
# label also has to be on GPU so .cuda() required -> this took an hour for me to figure out

cnn.eval()  
correct= 0
total = 0

for image, label in test_loader:
    image = Variable(image).cuda()
    result = cnn(image).cuda()
    _, predicted = torch.max(result.data, 1)
    total += label.size(0)
    correct += (predicted == label.cuda()).sum()

print('Test Accuracy of the model on the 10000 test images: %f %%' % (100 * correct / total))

Variable containing:
 2.3036
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
1.00000e-02 *
  7.4461
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.1331
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
1.00000e-02 *
  4.0274
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
1.00000e-02 *
  1.7283
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
1.00000e-02 *
  7.2050
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.3574
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
1.00000e-03 *
  3.9238
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
1.00000e-02 *
  1.8067
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.1295
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
1.00000e-02 *
  3.6189
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
1.00000e-02 *
  6.4373
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Vari